In [103]:
import os.path
from typing import List

import numpy as np
import pandas as pd

from datetime import date, datetime, timedelta

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_colwidth = 160

from utils.configuration import settings
from utils.utils import *
from tft.api import *


# Config

In [104]:
API_KEY: str = settings.api_key
ASSETS_DIR: str = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers' # challengers grandmasters
MAX_COUNT: int = settings.max_count
LATEST_RELEASE = '12.13.453.3037' # '12.12.450.4196' '12.13.453.3037' Version 12.12.448.6653 12.11.446.9344 Version 12.13.453.3037 (Jul 11 2022/18:39:20) [PUBLIC] <Releases/12.13>
RANKED_ID = 1100    # 1090 normal game 1100 ranked game
PATCH: date = date(2022, 7, 16)  # date(2022, 7, 1) date(2022, 7, 16)

TARGETNAME = 'placement'

In [105]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [106]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,_DZA8xlBLlJADXQ_-PmkDnCqKZlyyvGmYdScdL3dZxuuLDmO,ActUIBZZ8NtqCNDtUO0CAcLRZgXlTO0WAoAhP8ALkO63dsCE1CPIglXU,3GPeG8QlwBQmpBNPQVhBUR0G34lK4ppndL2YZVTBKAdvvoMAy47_WYA-4mUpRA1qDEBN3AH-cHnV7A,Emzeeshady TFT,25,1657471932000,1,_DZA8xlBLlJADXQ_-PmkDnCqKZlyyvGmYdScdL3dZxuuLDmO,Emzeeshady TFT,543,I,87,51,False,False,True,False
1,vmztnJjCXFVhgxGvBkrVHyYxFFHnfzRb-g6PKAzI2b-AyQWTzJOJtSf3KA,fFSxnxr6wunKRFBovX959e1enJ94EE1FzJP2mp8kQG4k7hlasGbKVxPk,sDlmpt3X5iVXreFo6nOZfChb6hmg02aVOpGu4LqcIBHcOpDIGGd-6riO0MkvLR1lE5IpXUx-Qrpwsw,zPacon,29,1658196838345,1,vmztnJjCXFVhgxGvBkrVHyYxFFHnfzRb-g6PKAzI2b-AyQWTzJOJtSf3KA,zPacon,294,I,91,67,False,False,True,False
2,oLnEvcKKmrxRCxD6SpX_6vUr97rSimC-XvMDvgqU9rpasV4,-Ulb_Wr4qgrgNQrrSsdQyJFB6p8jm8H-x1CMv2g4-CXWDA,c1p3orMcj4Mry3kIgdxT9DZHtcB9-UMhKCZgjODbWwNLIPc1WavkUckKzuZm6XGSAnnGeWeaf58Mbw,sweatypepega,717,1658198021000,70,oLnEvcKKmrxRCxD6SpX_6vUr97rSimC-XvMDvgqU9rpasV4,sweatypepega,359,I,113,94,False,False,True,True
3,Je1-OSJir7hbZG_ll4cEfBVsY3yik57RyqeRjI_u_q4FmtQ,EqaYqlbBjSudwsK4PPVYJLmmsPBtnPxOHHEBAW4-EsHr9AE,8Rv6zj9cB_Fsy3ypWndSvyzwWNAYXl9X__peDzYv7duhpmU-jORNSQ6LzlW9GMszF6cHej2evLh9Ug,Xiao Mei Bi,5415,1658205025857,303,Je1-OSJir7hbZG_ll4cEfBVsY3yik57RyqeRjI_u_q4FmtQ,Xiao Mei Bi,505,I,187,162,False,False,False,True
4,FzjWuFSJtv7j3V3AiwC6fsWlvsXqqGQdi0_ShYA1at0,9hj-3QOwGyJAqY3JpEXeOdtb_G-tJp5OMugu6Z60YQ,xQdeCZY8UxzF6WIhqEvqxiNEf8Smc0WdXKjU04_ZMAQ7PNHFvoCsw16OnfSonxk5uKPIgFX66c6dTA,MeNoHaxor,3582,1658211555000,245,FzjWuFSJtv7j3V3AiwC6fsWlvsXqqGQdi0_ShYA1at0,MeNoHaxor,526,I,184,160,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,y4TjJTRPsso49gHpm6nrSh5SX-Q6l4JvxWZpAPIvWcbbzKI,wMoC6ZnMBsK83VqC3Zr38-HaLDjBllJAopA8kTI6tRs6GTg,spYNLcJWlr9VNM1vD6peAfBbg_GPCTT9UPYLS2o3gq8fDWY_T6xF9ByVQCJL0Pfxgnf5KuWbX4s8BA,SirRavus,4492,1658181856000,269,y4TjJTRPsso49gHpm6nrSh5SX-Q6l4JvxWZpAPIvWcbbzKI,SirRavus,313,I,102,78,False,False,True,False
496,BuyEkluEGYqYkpYUMt0ePLXDg1yRnHTgMdtl0pdFrXbip20bPIN8BHPdAw,alGtvlyKMfJa03Na7r3nBE0amLThAhSwQhmfPIV4rilkKa2vNQxwsd7A,3cG9RtTPBg7JrOsKCRfsIRBlo-uz-3ejdBtMS8HQbCWj10YCOdwJjIxkBLcLVr9wxPN747cH8wmrjg,Captain Heemeow,5414,1658190426059,70,BuyEkluEGYqYkpYUMt0ePLXDg1yRnHTgMdtl0pdFrXbip20bPIN8BHPdAw,Captain Heemeow,383,I,139,118,False,False,True,True
497,DCuNUEjmCaDyVqBXmCDD3z100AfTFDx7P07fQz3ZQAk2X4U,Dj_P0NYby5Mr7nM3uM8954IvwhOLCVhIm5SmCKgTAaruEg,ZQedJujpP5NOWDjqsKHsVrJDAMR76osvTG4GzboPxYsQasCVkS3awMRJ6W8VvlYJIt7qYsuX9iuPFg,BeefCookies,1393,1658213496386,114,DCuNUEjmCaDyVqBXmCDD3z100AfTFDx7P07fQz3ZQAk2X4U,BeefCookies,454,I,202,164,False,False,True,False
498,l-h64Fiq8Wkvt-E1mIKQUnWVadYAE8ZATtICmriaED9j,5azTtjoSKNEvQSBr8lLFqI9ZSCFIS2CT7CcWhd_vzCU,PssjBGhGGMiVRrU7_jYf66chQStH6DPgk3ysTWKaxoE8W29IRS9HXWbP1HKepK2W9w1fWTdoTQn57A,deathbycow,539,1658139024000,34,l-h64Fiq8Wkvt-E1mIKQUnWVadYAE8ZATtICmriaED9j,deathbycow,288,I,115,96,False,False,True,False


# Load unique matches id

In [107]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [108]:
len(uniq_matches_id)

19544

In [109]:
len(matches_asset)

22967

In [110]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [111]:
len(uniq_matches)

19544

## Filter by LATEST_RELEASE version and RANKED_ID game.

In [112]:
matches_asset[-1]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [113]:
matches_asset[-1]['info']['queue_id']

1100

In [114]:
latest_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version']) and (
    RANKED_ID == match['info']['queue_id'])]


In [115]:
latest_patch_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                        and (PATCH <= date.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [116]:
timedelta(days=3)

datetime.timedelta(days=3)

In [117]:
latest_3d_matches = [match for match in uniq_matches if (LATEST_RELEASE in match['info']['game_version'])
                     and ((datetime.now() - timedelta(days=3)) <= datetime.fromtimestamp(match['info']['game_datetime']/1000.0))]


In [118]:
len(latest_matches)

6070

In [119]:
len(latest_patch_matches)

4716

In [120]:
len(latest_3d_matches)

3483

# Process api details to datasets rows

In [121]:
def process_matches(df) -> List:
    matches_array = []

    for match_row in df:
        match_id = match_row['metadata']['match_id']

        for participant in match_row['info']['participants']:
            match = {}
            match['match_id'] = match_id
            # match['level'] = participant['level']
            match['placement'] = participant['placement']
            # match['players_eliminated'] = participant['players_eliminated']
            # match['total_damage_to_players'] = participant['total_damage_to_players']

            for augment_index, augment in enumerate(participant['augments']):
                # if augment == 'TFT7_Augment_GuildLootHR':
                #     augment = 'TFT7_Augment_BandOfThieves1'
                match[f'augment{augment_index}'] = augment

            for trait_index, trait in enumerate(participant['traits']):
                match[f'{trait["name"]}'] = trait["tier_current"]

            for unit_index, unit in enumerate(participant['units']):
                match[f'{unit["character_id"]}'] = unit["tier"]
                match['TFT7_TrainerDragon_item1'] = 'None'
                match['TFT7_TrainerDragon_item2'] = 'None'
                for item_index, item in enumerate(unit['itemNames']):
                    match[f'{unit["character_id"]}_item{item_index}'] = item.split('_')[-1]

            matches_array.append(match)

    return matches_array


In [122]:
matches_array = process_matches(latest_matches)
matches_patch_array = process_matches(latest_patch_matches)
matches_3d_array = process_matches(latest_3d_matches)

In [123]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4378022832',
 'placement': 5,
 'augment0': 'TFT6_Augment_LudensEcho3',
 'augment1': 'TFT6_Augment_RadiantRelics',
 'augment2': 'TFT6_Augment_Diversify3',
 'Set7_Astral': 1,
 'Set7_Bruiser': 1,
 'Set7_Evoker': 0,
 'Set7_Guild': 1,
 'Set7_Legend': 0,
 'Set7_Mage': 2,
 'Set7_Mystic': 1,
 'Set7_Tempest': 0,
 'Set7_Trainer': 1,
 'Set7_Whispers': 0,
 'TFT7_Vladimir': 2,
 'TFT7_TrainerDragon_item1': 'None',
 'TFT7_TrainerDragon_item2': 'None',
 'TFT7_Heimerdinger': 2,
 'TFT7_Nami': 3,
 'TFT7_Nami_item0': 'ThiefsGloves',
 'TFT7_Nami_item1': 'SparringGloves',
 'TFT7_Nami_item2': 'Chalice',
 'TFT7_Lulu': 2,
 'TFT7_Ryze': 2,
 'TFT7_Ryze_item0': 'HextechGunblade',
 'TFT7_Ryze_item1': 'SpearOfShojin',
 'TFT7_Ryze_item2': 'MadredsBloodrazor',
 'TFT7_Sylas': 2,
 'TFT7_Sylas_item0': 'WarmogsArmor',
 'TFT7_Sylas_item1': 'WarmogsArmorRadiant',
 'TFT7_Sylas_item2': 'GargoyleStoneplate',
 'TFT7_Illaoi': 2,
 'TFT7_Ornn': 2,
 'TFT7_TrainerDragon': 1,
 'TFT7_TrainerDragon_item0': 'TrainerSn

In [124]:
matches_league_df = pd.json_normalize(matches_array)
matches_league_patch_df = pd.json_normalize(matches_patch_array)
matches_league_3d_df = pd.json_normalize(matches_3d_array)

In [125]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Astral,Set7_Bruiser,Set7_Evoker,Set7_Guild,Set7_Legend,...,TFT7_Gnar_item2,TFT7_LeeSin_item1,TFT7_LeeSin_item2,TFT7_Skarner_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Sett_item2,TFT7_Senna_item1,TFT7_Senna_item2,augment3
0,NA1_4378022832,5,TFT6_Augment_LudensEcho3,TFT6_Augment_RadiantRelics,TFT6_Augment_Diversify3,1.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4378022832,8,TFT6_Augment_LudensEcho3,TFT6_Augment_CelestialBlessing3,TFT7_Augment_MageEmblem2,0.0,1.0,0.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4378022832,1,TFT7_Augment_RevelEmblem2,TFT6_Augment_Diversify3,TFT6_Augment_RadiantRelics,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4378022832,4,TFT7_Augment_AssassinEmblem2,TFT7_Augment_ScalescornEmblem2,TFT6_Augment_CyberneticImplants3,NaN,1.0,NaN,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4378022832,3,TFT7_Augment_AssassinEmblem2,TFT6_Augment_TriForce3,TFT6_Augment_GrandGambler,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48555,NA1_4373229809,8,TFT7_Augment_ShimmerscaleTrait,TFT6_Augment_CyberneticImplants3,TFT7_Augment_RagewingEmblem,NaN,1.0,NaN,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48556,NA1_4373229809,4,TFT7_Augment_GuildEmblem,TFT6_Augment_ItemGrabBag2,TFT6_Augment_Distancing2,0.0,2.0,NaN,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48557,NA1_4373229809,7,TFT6_Augment_ThrillOfTheHunt2,TFT6_Augment_LudensEcho3,TFT7_Augment_MageEmblem,0.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48558,NA1_4373229809,3,TFT7_Augment_GadgetExpert,TFT7_Augment_UrfsGrabBag2,TFT6_Augment_Diversify2,NaN,NaN,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Astral,Set7_Bruiser,Set7_Evoker,Set7_Guild,Set7_Legend,...,TFT7_Gnar_item2,TFT7_LeeSin_item1,TFT7_LeeSin_item2,TFT7_Skarner_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Sett_item2,TFT7_Senna_item1,TFT7_Senna_item2,augment3
0,NA1_4378022832,5,TFT6_Augment_LudensEcho3,TFT6_Augment_RadiantRelics,TFT6_Augment_Diversify3,1.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4378022832,8,TFT6_Augment_LudensEcho3,TFT6_Augment_CelestialBlessing3,TFT7_Augment_MageEmblem2,0.0,1.0,0.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4378022832,1,TFT7_Augment_RevelEmblem2,TFT6_Augment_Diversify3,TFT6_Augment_RadiantRelics,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4378022832,4,TFT7_Augment_AssassinEmblem2,TFT7_Augment_ScalescornEmblem2,TFT6_Augment_CyberneticImplants3,NaN,1.0,NaN,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4378022832,3,TFT7_Augment_AssassinEmblem2,TFT6_Augment_TriForce3,TFT6_Augment_GrandGambler,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48555,NA1_4373229809,8,TFT7_Augment_ShimmerscaleTrait,TFT6_Augment_CyberneticImplants3,TFT7_Augment_RagewingEmblem,NaN,1.0,NaN,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48556,NA1_4373229809,4,TFT7_Augment_GuildEmblem,TFT6_Augment_ItemGrabBag2,TFT6_Augment_Distancing2,0.0,2.0,NaN,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48557,NA1_4373229809,7,TFT6_Augment_ThrillOfTheHunt2,TFT6_Augment_LudensEcho3,TFT7_Augment_MageEmblem,0.0,0.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48558,NA1_4373229809,3,TFT7_Augment_GadgetExpert,TFT7_Augment_UrfsGrabBag2,TFT6_Augment_Diversify2,NaN,NaN,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Sort and reorder columns

In [127]:
def reorder_df_col(df):
    fixed_cols = ['placement', 'match_id', 'augment0', 'augment1', 'augment2']
    all_cols = df.columns
    to_sort_cols = list(set(all_cols) - set(fixed_cols))

    return df.reindex(columns=fixed_cols + sorted(to_sort_cols))


In [128]:
matches_league_df = reorder_df_col(matches_league_df)
matches_league_patch_df = reorder_df_col(matches_league_patch_df)
matches_league_3d_df = reorder_df_col(matches_league_3d_df)

# Output dataframes

In [129]:
# matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)
# matches_league_patch_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.pickle'))
matches_league_patch_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{PATCH}_matches.csv'), index=False)
matches_league_3d_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")}_matches.pickle'))
matches_league_3d_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_{(datetime.now() - timedelta(days=3)).strftime("%Y-%m-%d")}_matches.csv'), index=False)

# End